In [42]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from textblob import TextBlob
import re
from sklearn.metrics import accuracy_score, classification_report


In [43]:
!pip install kagglehub

In [44]:
!pip install kaggle

In [45]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2


In [46]:
import os

# List all files in the directory where the dataset was downloaded
dataset_dir = "/root/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2"
for root, dirs, files in os.walk(dataset_dir):
    for name in files:
        print(os.path.join(root, name))


/root/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2/twitter_validation.csv
/root/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2/twitter_training.csv


In [47]:
import pandas as pd

# Load the dataset into a pandas DataFrame
dataset_path = '/root/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2/twitter_validation.csv'
df = pd.read_csv(dataset_path)

# Check the first few rows of the dataset and summary information
print(df.head())
print(df.info())


   3364   Facebook Irrelevant  \
0   352     Amazon    Neutral   
1  8312  Microsoft   Negative   
2  4371      CS-GO   Negative   
3  4433     Google    Neutral   
4  6273       FIFA   Negative   

  I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣  
0  BBC News - Amazon boss Jeff Bezos rejects clai...                                                                                                                                                                                                  
1  @Microsoft Why do I pay for WORD when it funct...                                                                                                                                                                                                  
2  CSGO matchmaking is so full of closet hacking,...        

In [48]:
print(df.columns)


Index(['3364', 'Facebook', 'Irrelevant',
       'I mentioned on Facebook that I was struggling for motivation to go for a run the other day, which has been translated by Tom’s great auntie as ‘Hayley can’t get out of bed’ and told to his grandma, who now thinks I’m a lazy, terrible person 🤣'],
      dtype='object')


In [49]:
def clean_text(text):
    # Remove mentions, hashtags, URLs, and special characters
    text = re.sub(r'@\w+|#\w+|http\S+|www\S+|[^A-Za-z0-9 ]+', '', text)
    # Convert text to lowercase
    text = text.lower()
    return text

# Apply text cleaning to the tweet column (assuming it’s the 4th column)
df['cleaned_text'] = df.iloc[:, 3].apply(clean_text)



In [50]:
# Update the sentiment column based on the values in 'Irrelevant' column
df['sentiment'] = df['Irrelevant'].map({'Positive': 1, 'Negative': 0, 'Neutral': 2})


In [51]:
X_train, X_test, y_train, y_test = train_test_split(
    df['cleaned_text'], df['sentiment'], test_size=0.2, random_state=42
)



In [52]:
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


In [53]:
print(y_train.isna().sum())  # This will print the count of NaN values in y_train


134


In [54]:
# Drop rows with NaN values in the 'sentiment' column
df_cleaned = df.dropna(subset=['sentiment'])

# Split the dataset again after dropping NaN values
X_train, X_test, y_train, y_test = train_test_split(
    df_cleaned['cleaned_text'], df_cleaned['sentiment'], test_size=0.2, random_state=42
)


In [55]:
# model = LogisticRegression()
# model.fit(X_train_vec, y_train)



# Drop rows where sentiment is NaN in both feature and target columns
df_cleaned = df.dropna(subset=['sentiment'])

# Now split the cleaned data
X_train, X_test, y_train, y_test = train_test_split(
    df_cleaned['cleaned_text'], df_cleaned['sentiment'], test_size=0.2, random_state=42
)

# Reapply vectorization after cleaning the data
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)



In [56]:
# print(len(X_train_vec), len(y_train))  # Both should have the same length

print(X_train_vec.shape[0], len(y_train))  # X_train_vec.shape[0] gives the number of rows


662 662


In [57]:
model = LogisticRegression()
model.fit(X_train_vec, y_train)  # Proceed with fitting the model


LogisticRegression()

In [58]:
# Make predictions on the test set
y_pred = model.predict(X_test_vec)


In [59]:
# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)



Accuracy: 0.5060240963855421


In [60]:
# Make predictions on the test set
y_pred = model.predict(X_test_vec)

# Calculate the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Check unique values in y_test to confirm number of classes
print("Unique values in y_test:", np.unique(y_test))

# Classification report for detailed performance metrics
print(classification_report(y_test, y_pred, target_names=['negative', 'neutral', 'positive']))


Accuracy: 0.5060240963855421
Unique values in y_test: [0. 1. 2.]
              precision    recall  f1-score   support

    negative       0.48      0.42      0.45        55
     neutral       0.71      0.49      0.58        65
    positive       0.40      0.63      0.49        46

    accuracy                           0.51       166
   macro avg       0.53      0.51      0.51       166
weighted avg       0.55      0.51      0.51       166

